In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import BaselineOnly, accuracy
from surprise.accuracy import rmse

In [16]:
#from sklearn.neighbors import NearestNeighbors
from numpy import array
from numpy.linalg import norm

from operator import itemgetter
import timeit

In [80]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import BaselineOnly, accuracy
from surprise.accuracy import rmse
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV

In [3]:
filepath = 'C:/Users/rl3079/Columbia/AppliedDataScience/prj4/'

In [32]:
full = pd.read_csv('full.csv')

In [33]:
full.head()

,Unnamed: 0,userId,movieId,rating,timestamp
0,0,1,1,3.0,964982703
1,1,1,3,3.0,964981247
2,2,1,6,3.0,964982224
3,3,1,44,4.0,964983815
4,4,1,47,4.0,964982931


In [34]:
full_data = full[['userId', 'movieId', 'rating']]

In [35]:
full_data.head()

,userId,movieId,rating
0,1,1,3.0
1,1,3,3.0
2,1,6,3.0
3,1,44,4.0
4,1,47,4.0


In [36]:
#train_np = train[['userId','movieId','rating']].values
#train_np = train_np.astype(np.int)
#test_np = test[['userId','movieId','rating']].values
#test_np = test_np.astype(np.int)

In [37]:
num_users = len(full.userId.unique())
num_movies = len(full.movieId.unique())
#num_movies = full['movieId'].unique().max()
print(num_users)
print(num_movies)

610
9724


In [38]:
len(full_data)

100836

In [39]:
reader = Reader()
data = Dataset.load_from_df(full_data, reader)

In [40]:

train, test = train_test_split(data, test_size = 0.2)

In [41]:

iterator = train.all_ratings()
new_df = pd.DataFrame(columns=['uid', 'iid', 'rating'])
i = 0
for (uid, iid, rating) in iterator:
    new_df.loc[i] = [uid, iid, rating]
    i = i+1

new_df.head(2)

,uid,iid,rating
0,0.0,0.0,4.0
1,0.0,63.0,3.0


In [42]:

min(new_df.rating.unique())

0.5

In [43]:
new_df['uid'] = new_df['uid'] + 1
new_df['iid'] = new_df['iid'] + 1

In [44]:

train_df = new_df
train_df.head()

,uid,iid,rating
0,1.0,1.0,4.0
1,1.0,64.0,3.0
2,1.0,92.0,3.0
3,1.0,99.0,3.5
4,1.0,119.0,2.5


In [45]:

test_df = pd.DataFrame(test, columns = ['uid', 'iid', 'rating'])

In [46]:
train_df.to_csv('train_split_by_surprise.csv')
test_df.to_csv('test_split_by_surprise.csv')

In [47]:
min(test_df.rating.unique())

0.0

In [48]:
len(test) + len(new_df)

100836

In [49]:

train_np = train_df[['uid','iid','rating']].values
train_np = train_df.astype(np.int)
test_np = test_df[['uid','iid','rating']].values
test_np = test_df.astype(np.int)

In [50]:

len(new_df)

80668

In [51]:
def format_output(I,M,U):
    approximation = np.multiply(I,np.dot(U.T,M))
    with open('output.txt','w') as f:  
        for i in range(610): # users
            for j in range(9724): # movies
                if approximation[i,j] != 0:
                    f.write("%d\t%d\t%f\n"%(i+1,j+1,approximation[i,j]))

In [52]:

def rmse(I,R,M,U,round=False):
    return np.sqrt(np.sum(np.square(np.multiply(I, (R-np.dot(U.T,M)))))/np.count_nonzero(I))

In [53]:
train_mat = train_np[['uid', 'iid', 'rating']].values
test_mat = test_np[['uid', 'iid', 'rating']].values

In [54]:
train_mat[0]

array([1, 1, 4])

In [55]:
R = np.asmatrix(np.zeros((num_users,num_movies), dtype=np.int)) # sparse rating matrix
for row in train_mat:
    user = row[0] - 1  # users start with id 1
    movie = row[1] - 1
    rating = row[2]
    R[user, movie] = rating

In [56]:
Rtst = np.asmatrix(np.zeros((num_users,num_movies), dtype=np.int))
for row in test_mat:
    user = row[0] - 1
    movie = row[1] - 1
    rating = row[2]
    Rtst[user,movie] = rating

In [57]:
# Indicator Matrices
I = np.copy(R)
I[I > 0] = 1
I[I == 0] = 0
I2 = np.copy(Rtst)
I2[I2 > 0] = 1
I2[I2 == 0] = 0

In [58]:
# ALS Parameters
l = 0.1       # Regularization parameter lambda
k = 0         # convergence iterator
K = 5        # convergence criterion
r = 10      # Dimensionality of latent feature space
m,n = R.shape # Number of users and items

In [65]:
np.random.seed(0)
np.seterr(divide='ignore', invalid='ignore') # supress divide by zero warning

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [67]:
# Initialize matricies
U = 1 * np.random.rand(r,m) # Latent user feature matrix (pattern matrix)
M = 1 * np.random.rand(r,n) # Latent movie feature matrix (coefficient matrix)
avg_movies = np.divide(R.sum(0),(R!=0).sum(0))
avg_movies[np.isnan(avg_movies)] = 0
M[0,:] = avg_movies # Set first row of Q to column vector of average ratings
E = np.eye(r,dtype=int) # rxr idendity matrix

In [68]:
train_errors = []
test_errors = []

In [69]:
# Repeat until convergence
while k < K:
    # Fix M and solve for U
    for i, Ii in enumerate(I):
        nui = np.count_nonzero(Ii) # Number of items user i has rated
        if (nui == 0): nui = 1 # remove zeros
    
        # Least squares solution
        Ai = np.add(np.dot(M, np.dot(np.diag(Ii), M.T)), np.multiply(l, np.multiply(nui, E))) # A_i = M_{I_i}M_{I_i}^T + ln_{u_i}E
        Vi = np.dot(M, np.dot(np.diag(Ii), R[i].T)) # V_i = M_{I_i}R^T(i,I_i)
        U[:,i] = np.linalg.solve(Ai,Vi).reshape(r)

    # Fix U and solve for M
    for j, Ij in enumerate(I.T):
        nmj = np.count_nonzero(Ij) # Number of users that rated item j
        if (nmj == 0): nmj = 1 # remove zeros

        # Least squares solution
        Aj = np.add(np.dot(U, np.dot(np.diag(Ij), U.T)), np.multiply(l, np.multiply(nmj, E))) # A_j = U_{I_j}U_{I_j}^T + ln_{m_j}E
        Vj = np.dot(U, np.dot(np.diag(Ij), R[:,j])) # V_j = U_{I_j}R(I_j,j)
        try:
            M[:,j] = np.linalg.solve(Aj,Vj).reshape(r)
        except: 
            print("singular matrix error: " + str(j + 1)) # movie has no reviewers 
            continue

    train_rmse = rmse(I,R,M,U)
    test_rmse = rmse(I2,Rtst,M,U)
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)
    #print "[k: %d/%d] train-RMSE = %f  test-RMSE = %f" %(k+1, K, train_rmse, test_rmse)
    k = k + 1

format_output(I2,M,U)
print(test_errors)

[1.4660923250868945, 1.5480692545317587, 1.589313241279779, 1.6009771894594715, 1.6010275136499892]


In [73]:
print ("[k: %d/%d] train-RMSE = %f  test-RMSE = %f" %(k+1, K, train_rmse, test_rmse))

[k: 6/5] train-RMSE = 0.618053  test-RMSE = 1.601028


In [77]:
cols = ['movieId', 'userId', 'rating']
ratings = pd.read_csv('ratings.csv', usecols = cols)

In [78]:
reader = Reader()
data = Dataset.load_from_df(ratings, reader)

In [79]:
train, test = train_test_split(data, test_size = 0.2)

In [82]:
param_grid = {'k': [1],
              'n_epochs':[5, 10],
              'bsl_options': {'method': ['als'],
                              'reg_u': [12, 13]},
              'sim_options': {'name': ['cosine'],
#                               'min_support': [1, 5],
                              'user_based': [False]}
              }

gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv = 5)

gs.fit(data)
print(gs.best_score['rmse'])
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

In [83]:
print(gs.best_score['rmse'])

1.3328526313948927


In [84]:
results_df = pd.DataFrame.from_dict(gs.cv_results)

In [85]:
predictions = algo.test(test)

In [86]:
predictions

[Prediction(uid=480, iid=4310, r_ui=2.5, est=3.5, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=119, iid=104841, r_ui=3.5, est=4.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=313, iid=1580, r_ui=4.0, est=4.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=474, iid=3966, r_ui=3.5, est=4.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=305, iid=45720, r_ui=3.5, est=4.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=318, iid=32906, r_ui=3.5, est=3.5, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=573, iid=60684, r_ui=5.0, est=5, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=542, iid=8958, r_ui=5.0, est=3.5, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=156, iid=2186, r_ui=4.0, est=2.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=292, iid=86817, r_ui=2.5, est=3.5, details={'actual_k': 1, 'was_impossib

In [87]:
rmse(predictions, verbose = True)

RMSE: 1.1196


1.1196460152762562

In [92]:
print(gs.best_params['rmse'])

{'k': 1, 'n_epochs': 5, 'bsl_options': {'method': 'als', 'reg_u': 12}, 'sim_options': {'name': 'cosine', 'user_based': False}}


In [93]:
def cosine_similarity_all(test_movie_id):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,int. for testing purpose should also match same user
    Return
    
    ----
        """
  
    ###########
    movie_test_id_str_list = list(str(test_movie_id))

    movie_compare_id = [item for item in movie_id if item not in movie_test_id_str_list] #movie_id list without test id
    ###############

    l_latent_movie = []
    movie_cos_dic = {}
    for i in movie_compare_id:
       # print(i,type(i)) 2，str

        numerator =array([val
                      for sublist in np.asarray(q[[movie_test_id_str_list[0]]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[movie_test_id_str_list[0]]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)


        dict_movieid_cos = dict(zip(movie_compare_id,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1)) #from small to large

    return ([ele for ele in reversed(l_sorted_s_to_l)])

In [94]:
def knn_k_pq(test_movie_id,df,userid):
    """
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        df: df_mat（only pq)
        userid： int
    Return
    -----
        a list of  predicted rating for given movie id for given user given year of matrix
        
        """
    distance_pair = cosine_similarity_all(test_movie_id)

    movie_index = b_item_dic.get(distance_pair[0][0])
    user_index = b_user_dic.get(str(userid))
    
    rating_predict = df[user_index][movie_index]
    
    return round(rating_predict,1)